In [2]:
from api.filters import get_filter_query
import requests
from pprint import pprint

In [3]:
filters = {
    'numerical_range': [
        {
            'attribute_name': 'price',
            'min': None,
            'max': None
        },
        {
            'attribute_name': 'accommodates',
            'min': None,
            'max': None
        },
#         {
#             'attribute_name': 'id',
#             'min': 109,
#             'max': 109
#         }
    ],
    'region': {
        'region_type': 'neighborhood',
        'id': Neighborhood.objects.get(name="Eagle Rock").pk
    }
}
query = get_filter_query(filters)
print(query)

(AND: ('neighborhood', 76))


In [4]:
%%time
r=requests.post('http://reaction/api/listings/stats/', json={'filters': filters})
stats=r.json()
print(stats['listings']['most_discriminating_terms']['value'])

['eagle', 'rock', 'pasadena', 'highland', 'occidental', 'house', 'glendale', 'lots', 'yard', 'college', 'park', 'trees', 'entrance', 'town', 'rose', 'joes', 'trader', 'bowl', 'questions', 'colorado']
CPU times: user 6.96 ms, sys: 2.48 ms, total: 9.44 ms
Wall time: 3.67 s


In [158]:
Neighborhood.objects.first().fixed_data

{'median_age': 45.1756062767475,
 'median_gross_rent': 670.6654991243432,
 'median_home_value_owner_occupied': 415123.50262697024,
 'median_household_income': 83798.29791631277,
 'per_capita_income': 34437.263908701854,
 'percent_associate_degree': 0.10245824732595235,
 'percent_bachelors_degree': 0.16513417151435544,
 'percent_doctoral_degree': 0.013698630136986302,
 'percent_homes_vacant': 0.09737302977232926,
 'percent_masters_degree': 0.08707074498029649,
 'percent_professional_degree': 0.012385062863576657,
 'population_latimes_2000_census': 6522}

In [1]:
for n in Neighborhood.objects.all():
    n.update_stats()

In [3]:
Neighborhood.objects.first().computed_stats

{'avg_estimated_bookings_per_listing_per_month': 4.34049479166666,
 'avg_estimated_revenue_per_listing_per_month': 5307.615234375,
 'avg_host_experience_years': 0.543310082435003,
 'avg_listing_price': 451.0,
 'crime_count': 842,
 'crimes_per_capita': 0.1291015026065624,
 'listing_count': 3,
 'total_estimated_bookings_per_month': 6.5107421875}

In [16]:
listing_attrs = {
    'address': 'Pacific Promenade, Playa Vista, CA',
    'availability_365': 365,
    'room_type': 'Private room',
    'property_type': 'Apartment',
    'bed_type': 'Real Bed',
    'accommodates': 2,
    'guests_included': 2,
    'host_experience_days': 360,
    'extra_people': 0,
    'review_count': 0,
    'minimum_nights': 1,
    'amenity_1': False,
    'amenity_2': False,
    'amenity_3': False,
    'amenity_4': False,
    'amenity_5': False,
    'amenity_6': True,
    'amenity_7': False,
    'amenity_8': False,
    'amenity_9': False,
    'amenity_10': False,
    'amenity_11': False,
    'amenity_12': False,
    'amenity_13': False,
    'amenity_14': False,
    'amenity_15': False,
    'amenity_16': False,
    'amenity_17': False,
    'amenity_18': False,
    'amenity_19': False,
    'amenity_20': False,
    'amenity_21': False,
    'amenity_22': False,
    'amenity_23': False,
    'amenity_24': False,
    'amenity_25': False,
    'amenity_26': False,
    'amenity_27': False,
    'amenity_28': False,
    'amenity_29': False,
    'amenity_30': False,
    'amenity_31': False,
    'amenity_32': False,
    'amenity_33': False,
    'amenity_34': False,
    'amenity_35': False,
    'amenity_36': False,
    'amenity_37': False,
    'amenity_38': False,
    'amenity_39': False,
    'amenity_40': False,
    'amenity_41': False,
    'amenity_42': False,
    'amenity_43': False,
    'amenity_44': False,
}
r=requests.post('http://localhost:8000/api/predict/', json={'listing_attrs': listing_attrs})

In [17]:
r.content

b'{"predicted_price":127.36666666666666}'